# SQL Joins

We talked about joining tables together in Module 1. In this lecture, we are going to revisit this concept again.  
![Joins](../assets/sql-joins.png)


- LEFT JOIN returns all records from the left table and records from the right table that are found in the left table
- RIGHT JOIN returns all records from the right table and records from the left table that are found in the right table (not supported in SQlite)
- INNER JOIN only returns records that are present in both tables
- FULL JOIN returns all records from both tables (not supported in SQlite)


When joining tables together, it's important to understand different types of relationships. 

- One-to-one
- One-to-many
- Many-to-many

Here is standard SQL syntax for joining tables together

<code> SELECT a.col1, a.col2, b.col3, b.col4
       FROM table1 a
       _LEFT_/_RIGHT_/_INNER_/_FULL_ JOIN table2 b
       ON a.col1 = b.col2 </code>

More than two tables can be joined in SQL. Column names used to join tables don't need to be the same (unlike Pandas). You can also join on more than two columns

<code> SELECT a.col1, a.col2, b.col3, b.col4
       FROM table1 a
       _LEFT_/_RIGHT_/_INNER_/_FULL_ JOIN table2 b
       ON a.col1 = b.col2 and a.col3 = b.col3 </code>


In [ ]:
import pandas as pd
import sqlite3

In [ ]:
# create sqlite db
info = pd.read_csv("../data/customer-info.csv")
loyalty = pd.read_csv("../data/customer-loyalty.csv")
sales2019 = pd.read_csv("../data/sales2019.csv")
sales2020 = pd.read_csv("../data/sales2020.csv")
sales2021 = pd.read_csv("../data/sales2021.csv")
zipcode = pd.read_csv("../data/Zipcode-ZCTA-Population-Density-And-Area-Unsorted.csv")
conn = sqlite3.connect('../data/generalstore.sqlite') 
info.to_sql('info',con=conn,index=False,if_exists='replace')
loyalty.to_sql('loyalty',con=conn,index=False,if_exists='replace')
sales2019.to_sql('sales2019',con=conn,index=False,if_exists='replace')
sales2020.to_sql('sales2020',con=conn,index=False,if_exists='replace')
sales2021.to_sql('sales2021',con=conn,index=False,if_exists='replace')
zipcode.to_sql('zipcode',con=conn,index=False,if_exists='replace')

In [ ]:
#Returns the table name. Including ones that we have used previously. 

pd.read_sql("SELECT name FROM sqlite_master WHERE type='table'",con=conn)

In [ ]:
pd.read_sql("""SELECT *
                FROM info
                LIMIT 5""", conn)

In [ ]:
pd.read_sql("""SELECT *
                FROM zipcode
                LIMIT 5""", conn) # not ideal naming convention

In [ ]:
## Left join info with zip code
pd.read_sql("""
            SELECT t1.*, t2.*
            FROM info t1
            LEFT JOIN zipcode t2
            ON t1.ZipCode = t2.[Zip/ZCTA]
            """, conn)

In [ ]:
## Inner join info with zip code
pd.read_sql("""
            SELECT t1.*, t2.*
            FROM info t1
            JOIN zipcode t2
            ON t1.ZipCode = t2.[Zip/ZCTA]
            """, conn)

In [ ]:
# One to Many join
pd.read_sql("""
            SELECT * 
            FROM info t1
            LEFT JOIN Sales2019 t2
            ON t1.CustomerID = t2.CustomerID""", conn) 

In [ ]:
#One to One join
pd.read_sql("""
            SELECT * 
            FROM info t1
            LEFT JOIN (
                    SELECT CUSTOMERID, sum(Sales2019) as Sales2019, count(item_description) as itemcnt
                    FROM Sales2019
                    GROUP BY CUSTOMERID) t2
            ON t1.CustomerID = t2.CustomerID""", conn) 

In [ ]:
# # show customer id from 1001 through 1010 - Will fail
pd.read_sql("""
            SELECT * 
            FROM info t1
            LEFT JOIN (
                    SELECT CUSTOMERID, SUM(Sales2019) AS Sales2019, COUNT(item_description) AS itemcnt
                    FROM Sales2019
                    GROUP BY CUSTOMERID) t2
            ON t1.CustomerID = t2.CustomerID
            WHERE CustomerID between 1001 and 1010""", conn) # ambiguous Column Name

In [ ]:
# show customer id from 1001 through 1010
pd.read_sql("""
            SELECT * 
            FROM info t1
            LEFT JOIN (
                    SELECT CUSTOMERID, SUM(Sales2019) AS Sales2019, count(item_description) AS itemcnt
                    FROM Sales2019
                    GROUP BY CUSTOMERID) t2
            ON t1.CustomerID = t2.CustomerID
            WHERE t1.CustomerID between 1001 and 1010""", conn) 

In [ ]:
## ADDING WHERE, GROUP BY, and ORDER BY

# show 2021 sales by Occupation outside of NY, sort by sales in desc order
pd.read_sql("""
            SELECT Occupation, sum(Sales2021) as Sales
            FROM info t1
            LEFT JOIN (
                    SELECT CUSTOMER_ID, SUM(Sales2021) AS Sales2021, COUNT(item_description) AS itemcnt
                    FROM Sales2021
                    GROUP BY CUSTOMER_ID) t2
            ON t1.CustomerID = t2.Customer_ID
            WHERE State <> 'NY'
            GROUP BY Occupation
            ORDER BY Sales desc""", conn) 

# UNION

UNION can also combine two or more tables but unlike JOIN, it appends tables on top of each other. In order for UNION to work correctly, you need to specify the same number of columns appearing in the same order from each table. The columns should also be of similar data type.

When can UNION be helpful? If you have similar data stored in different tables. Perhaps, data from one year is stored in one table whereas data from another year is stored in a different table, and you want to combine it in one table, that's when you would use the UNION operator.

## UNION vs UNION ALL

UNION: keeps unique records
UNION ALL: keeps all records

In [ ]:
#One to One join - will fail
pd.read_sql("""
            SELECT * FROM Sales2019
            UNION
            SELECT * from Sales2020""", conn) 

In [ ]:
pd.read_sql("""

            SELECT CUSTOMERID, Sales2019, item_description, Date FROM Sales2019
            UNION ALL
            SELECT customer_id, Sales2020, item_description, YearMonth FROM Sales2020""", conn)

## Union ALL and Union -> same number of records but sorted differently

In [ ]:
pd.read_sql("""

            SELECT CUSTOMERID, Sales2019, item_description, Date FROM Sales2019
            UNION ALL
            SELECT customer_id, Sales2020, item_description, YearMonth FROM Sales2020
            UNION ALL
            SELECT customer_id, Sales2021, item_description, YearMonth FROM Sales2021""", conn)

In [ ]:
## Clean output

pd.read_sql("""

            SELECT CUSTOMERID AS customer_id, 
                    Sales2019 AS Sales, 
                    item_description, 
                    substr(Date,7,4)||substr(Date,1,2) AS YearMonth
            FROM Sales2019
                UNION ALL
            SELECT customer_id, 
                    Sales2020 AS Sales, 
                    item_description, 
                    YearMonth 
            FROM Sales2020
                UNION ALL
            SELECT customer_id, 
                    Sales2021 AS Sales, 
                    item_description, 
                    YearMonth 
            FROM Sales2021
            ORDER BY customer_id, YearMonth""", conn)

# Fuzzy Matching

Sometimes you may have two tables that need to be joined on a string instead of an ID. This can be very painful! 

Certain variations of SQL have built-in functions that allow you to check how similar two strings are, for example, COMPGED calculates the Levenshtein distance (more on this in NLP) or the SOUNDEX function. Fuzzy matching isn't supported in SQLite

[Fuzzywuzzy](https://pypi.org/project/fuzzywuzzy/) is a great Python alternative for fuzzy matching.

There are also NoSQL databases that are great for entity resolution ([Elastic Search](https://www.elastic.co/elastic-stack/)).
